# Baseline Results
Baseline models are:
- bert-large-uncased
- roberta-large
- mpnet-base

In [5]:
import pandas as pd
import sys
sys.path.append("..")
from baseline.parse_output import get_results_overall_table, get_results_table, parse_preds_baseline
from src.Utils import parse_preds

MODELS = ["bert-large-uncased", "roberta-large", "mpnet-base"]
BASE_PATH = "../baseline/music-ner-eacl2023/output/shs100k2/complete"


In [6]:
get_results_table(MODELS)


f1                                          \
Attribute             Artist                           WoA             
Scenario               exact    strict      type     exact    strict   
Model                                                                  
bert-large-uncased  0.904479  0.899604  0.916318  0.912811  0.904708   
mpnet-base          0.898158  0.892833  0.911550  0.921381  0.916022   
roberta-large       0.903440  0.898807  0.915937  0.915952  0.909427   

                             precision                                \
Attribute                       Artist                           WoA   
Scenario                type     exact    strict      type     exact   
Model                                                                  
bert-large-uncased  0.943012  0.893328  0.888513  0.905021  0.907101   
mpnet-base          0.951328  0.890518  0.885239  0.903797  0.916012   
roberta-large       0.946490  0.894549  0.889962  0.906923  0.911202   

                                          recall                      \
Attribute                                 Artist                       
Scenario              strict      type     exact    strict      type   
Model                                                                  
bert-large-uncased  0.899048  0.937113  0.915912  0.910975  0.927901   
mpnet-base          0.910683  0.945784  0.905930  0.900559  0.919438   
roberta-large       0.904711  0.941582  0.912509  0.907830  0.925132   

                                                  
Attribute                WoA                      
Scenario               exact    strict      type  
Model                                             
bert-large-uncased  0.918593  0.910439  0.948986  
mpnet-base          0.926814  0.921423  0.956938  
roberta-large       0.920752  0.914193  0.951450

In [7]:
get_results_overall_table(MODELS)


macro                                              \
Scenario              exact                                               
Metric               actual  correct        f1 incorrect missed partial   
Model                                                                     
bert-large-uncased  16963.0  15257.0  0.908645     442.0  937.0     0.0   
mpnet-base          16883.0  15226.0  0.909770     437.5  972.5     0.0   
roberta-large       16846.0  15194.0  0.909696     434.5  955.5     0.0   

                                                          ...    micro  \
Scenario                                                  ...     type   
Metric             possible precision    recall spurious  ...   actual   
Model                                                     ...            
bert-large-uncased  16636.0  0.900214  0.917252   1264.0  ...  33926.0   
mpnet-base          16636.0  0.903265  0.916372   1219.5  ...  33766.0   
roberta-large       16584.0  0.902875  0.916630   1217.5  ...  33692.0   

                                                                          \
Scenario                                                                   
Metric              correct        f1 incorrect  missed partial possible   
Model                                                                      
bert-large-uncased  31186.0  0.928182     212.0  1874.0     0.0  33272.0   
mpnet-base          31148.0  0.929264     179.0  1945.0     0.0  33272.0   
roberta-large       31074.0  0.929524     183.0  1911.0     0.0  33168.0   

                                                 
Scenario                                         
Metric             precision    recall spurious  
Model                                            
bert-large-uncased  0.919236  0.937305   2528.0  
mpnet-base          0.922466  0.936163   2439.0  
roberta-large       0.922296  0.936867   2435.0  

[3 rows x 60 columns]

# Error Analysis
Based on [Batista (2018)](https://www.davidsbatista.net/blog/2018/05/09/Named_Entity_Evaluation/), we can have three types of errors:
- missing entities (False Negatives)
- spurious entities (False Positives)
- incorrect entities (only partially detected entities)


In [8]:
data = pd.read_parquet("../data/intermediate/shs100k2_IOB.parquet")
data = data.loc[(data.split == "TEST") & (data.part.isin(["both_100", "medium"])), 
         ["set_id", "yt_id", "title", "performer", "TEXT", "IOB"]]

for model in MODELS:
    data["IOB", model] = parse_preds_baseline(model)


In [9]:
data

,set_id,yt_id,title,performer,TEXT,IOB,"(IOB, bert-large-uncased)","(IOB, roberta-large)","(IOB, mpnet-base)"
0,21,gORyrU1xQpg,[Yesterday],[marianne faithfull],"[marianne, faithfull, -, yesterday, (, with, l...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ..."
1,21,jQhC3bMMLmw,[Yesterday],[matt monro],"[matt, monro, -, yesterday, ., nancyfloressant...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ..."
4,21,eI3-EVEU51s,[Yesterday],"[the hollyridge strings, hollyridge strings]","[hollyridge, strings, -, yesterday, ., boyjohn...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ..."
5,21,ew1y-gvO_NM,[Yesterday],[cilla black],"[cilla, black, :, :, :, :, :, yesterday, ., .,...","[B-Artist, I-Artist, O, O, O, O, O, B-WoA, O, ...","[B-Artist, I-Artist, O, O, O, O, O, B-WoA, O, ...","[B-Artist, I-Artist, O, O, O, O, O, B-WoA, O, ...","[B-Artist, I-Artist, O, O, O, O, O, B-WoA, O, ..."
6,21,tjYQuEyxRp0,[Yesterday],[sarah vaughan],"[sarah, vaughan, -, yesterday, ., david, speed...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ...","[B-Artist, I-Artist, O, B-WoA, O, O, O, O, O, ..."
...,...,...,...,...,...,...,...,...,...
10539,9993,3qcMjKxt-dE,[53rd & 3rd],[ramones],"[ramones, -, 53rd, &, 3rd, ., the, ramones, ♪,...","[B-Artist, O, B-WoA, I-WoA, I-WoA, O, O, B-Art...","[B-Artist, O, B-WoA, I-WoA, I-WoA, O, O, B-Art...","[B-Artist, O, B-WoA, I-WoA, I-WoA, O, O, B-Art...","[B-Artist, O, B-WoA, I-WoA, I-WoA, O, O, B-Art..."
10541,9993,P27xcT6mJP4,[53rd & 3rd],[metallica],"[53rd, &, 3rd, -, metallica, ., funinfuneral13...","[B-WoA, I-WoA, I-WoA, O, B-Artist, O, O, O, O,...","[B-WoA, I-WoA, I-WoA, O, B-Artist, O, O, O, O,...","[B-WoA, I-WoA, I-WoA, O, B-Artist, O, O, O, O,...","[B-WoA, I-WoA, I-WoA, O, B-Artist, O, O, O, O,..."
10543,9994,EVsRN0eDGNc,[Sytten ti],[otto brandenburg],"[otto, brandenburg, -, sytten, ti, ., ., ., ch...","[B-Artist, I-Artist, O, B-WoA, I-WoA, O, O, O,...","[B-Artist, I-Artist, O, B-WoA, I-WoA, O, O, O,...","[B-Artist, I-Artist, O, B-WoA, I-WoA, O, O, O,...","[B-Artist, I-Artist, O, B-WoA, I-WoA, O, O, O,..."
10544,9998,QveJv7V4OZI,[I Don't Care],[ramones],"[ramones, -, i, dont, care, ., the, ramones, ♪...","[B-Artist, O, B-WoA, I-WoA, I-WoA, O, O, B-Art...","[B-Artist, O, B-WoA, I-WoA, I-WoA, O, O, B-Art...","[B-Artist, O, B-WoA, I-WoA, I-WoA, O, O, B-Art...","[B-Artist, O, B-WoA, I-WoA, I-WoA, O, O, B-Art..."


In [10]:
from preprocessing.Utils import B_PREFIX, I_PREFIX, O_LABEL, overlap
from typing import List, Dict, Tuple
from src.Utils import get_missing_ents, get_spurious_ents, get_incorrect_ents

for model in MODELS:
    data[("missing", model)] = data.apply(lambda row: get_missing_ents(row["IOB"], row[("IOB", model)]), axis=1)
    data[("spurious", model)] = data.apply(lambda row: get_spurious_ents(row["IOB"], row[("IOB", model)]), axis=1)
    data[("incorrect", model)] = data.apply(lambda row: get_incorrect_ents(row["IOB"], row[("IOB", model)]), axis=1)


# Error Analyse Aspekte:
- Artist und Titel gruppieren
    - nach Länge
    - nach POS Tag 
    - "feat"
    - Popularity pro Artist
        - Wikidata
        - Views
    - Sprache der Wörter?
        - WordNet
            - Sind Namen und Artists im WordNet
    - Top-Down vs. Bottom Up
        - Bottom Up: Error Analyse -> ändern

# Manual Result Computation

In [12]:
path = "../output/llama3_ner_zeroshot.jsonl"
parse_preds(path)


[array(['Artist', 'WoA', 'O', 'WoA', 'O', 'O', 'O', 'O', 'O', 'WoA', 'O',
        'O', 'O', 'O', 'O', 'O', 'O'], dtype=object),
 array(['O', 'O', 'O', 'WoA', 'O', 'Artist', 'O', 'O', 'O', 'WoA', 'O',
        'O', 'O', 'O'], dtype=object),
 array(['O', 'O', 'O', 'WoA', 'O', 'Artist', 'O', 'O', 'O', 'O', 'O', 'O',
        'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
        'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Artist', 'O', 'O', 'O', 'O',
        'O', 'O', 'O', 'O', 'Artist'], dtype=object),
 array(['O', 'O', 'O', 'WoA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
        'O', 'Artist', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
        'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
        'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
        'O'], dtype=object),
 array(['O', 'O', 'O', 'WoA', 'O', 'Artist', 'O', 'O', 'O', 'O', 'O', 'O',
        'O', 'Artist', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
        'O', 'O

In [13]:
import importlib
import sys
sys.path.append('../baseline/music-ner-eacl2023/music-ner/src')
eval_utils = importlib.import_module("eval_utils")
from eval_utils import compute_results

mask = (data["IOB"].apply(len) == data["IOB", "roberta-large"].apply(len))
matching_data = data[mask]
compute_results(matching_data.IOB.to_list(), matching_data["IOB", "roberta-large"].to_list())


2024-06-19 07:24:36 root INFO: Imported 7725 predictions for 7725 true examples



 Overall
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.897494  0.915266  0.906293      27501          993          0      1553        2148       30047     30642
ent_type     0.924254  0.942557  0.933316      28321          173          0      1553        2148       30047     30642
exact        0.903139  0.921024  0.911994      27674          820          0      1553        2148       30047     30642

 Artist
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.890492  0.915682  0.902912      15117          392          0      1000        1467       16509     16976
ent_type     

{'overall_strict_precision_micro': 0.8974936361856276,
 'overall_strict_recall_micro': 0.9152660831364197,
 'overall_strict_f1_micro': 0.9062927383875167,
 'overall_strict_correct_micro': 27501,
 'overall_strict_incorrect_micro': 993,
 'overall_strict_partial_micro': 0,
 'overall_strict_missed_micro': 1553,
 'overall_strict_spurious_micro': 2148,
 'overall_strict_possible_micro': 30047,
 'overall_strict_actual_micro': 30642,
 'overall_ent_type_precision_micro': 0.9242542914953332,
 'overall_ent_type_recall_micro': 0.9425566612307386,
 'overall_ent_type_f1_micro': 0.9333157573860172,
 'overall_ent_type_correct_micro': 28321,
 'overall_ent_type_incorrect_micro': 173,
 'overall_ent_type_partial_micro': 0,
 'overall_ent_type_missed_micro': 1553,
 'overall_ent_type_spurious_micro': 2148,
 'overall_ent_type_possible_micro': 30047,
 'overall_ent_type_actual_micro': 30642,
 'overall_exact_precision_micro': 0.9031394817570655,
 'overall_exact_recall_micro': 0.921023729490465,
 'overall_exact_f1